In [5]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyreadstat
import re
import string
import sklearn
from sklearn_pandas import DataFrameMapper
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, LogisticRegression,LogisticRegressionCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR


from tqdm import tqdm_notebook as tqdm
from reed import *
#from cinspect import dependence, importance
from sklearn.model_selection import cross_val_score, cross_validate

from econml.orf import DMLOrthoForest, DROrthoForest
from econml.dml import CausalForestDML
from econml.sklearn_extensions.linear_model import WeightedLassoCVWrapper, WeightedLasso, WeightedLassoCV
from sklearn.linear_model import LogisticRegression, Lasso


# set global notebook options
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500
%matplotlib inline

%load_ext autoreload
%autoreload 2

import logging.config
DEFAULT_LOGGING = {
    'version': 1,
    'disable_existing_loggers': False,
    'loggers': {
        '': {
            'level': 'INFO',
        },
    }
}

logging.config.dictConfig(DEFAULT_LOGGING)

# Load the data

In [6]:
outcome = 'y_Dwsce'#'y_wsce'
treatment = 'redufl'
optimisation_metric = 'neg_mean_squared_error'
evaluation_metrics = ('r2','neg_mean_squared_error')
log_outcome=False
data_file = "all_vars.csv"

In [7]:
data = pd.read_csv(data_file,index_col='xwaveid')
drop_missing_treatment_or_outcome(data, treatment, outcome)
if log_outcome:
    data[outcome] = np.log(data[outcome]+data[outcome].min())

Dropped 592 rows missing treatment or outcome.


### Prepare data for modeling

In [24]:
from direct_regression import seperate_and_transform_data
X0, X1, y0, y1, X, y, T, features = seperate_and_transform_data(data, treatment, outcome)

print("Control data dimensions: ",X0.shape)
print("Treated data dimensions:",X1.shape)

Control data dimensions:  (3659, 638)
Treated data dimensions: (1295, 638)


### Basic check on E[Y] and E[Y|T]

In [25]:
from direct_regression import print_unconditional_effects
print_unconditional_effects(data, treatment, y0, y1)

Proportion Treated:26%
Average outcome under Control:67.85±15.36
Average outcome under Treatment:310.57±29.39
Unadjusted treatment estimate 242.72


### Separate covariates X into Xh and W

EconML requires `X` (covariates with which to compute heterogeneity) to be set, even if you only want to estimate ATE. 

We will use `Xh` for the covariates passed to $X$ for EconML and `W` for the remaining variables ie `Union(Xh, W) = X`

For initial testing, lets just take the feature with the largest coefficient to be the one to compute CATE with respect to. 

In [26]:
def find_most_predictive_feature(X,T,y,features):
    model = Lasso()
    V = np.hstack((X,T.reshape(-1,1)))
    features_ext = features + ['treatment']
    model.fit(V,y)
    coef = pd.DataFrame({"coef":model.coef_},index=features_ext)
    coef['magnitude'] = coef['coef'].abs()
    coef_mag = coef.sort_values('magnitude',ascending=False)  
    return coef_mag

coef_mag = find_most_predictive_feature(X,T,y,features)
coef_mag.head(3)

,coef,magnitude
awscmei,-377.779591,377.779591
ahgage,-352.846828,352.846828
ahiwsfei,119.712270,119.712270


In [27]:
hetero_feature = coef_mag.iloc[0].name
print(f"Computing treatment effect heterogeneity with respect to: {hetero_feature}")
x_indx = features.index(hetero_feature)
Xh = X[:,x_indx].reshape(-1,1) # X for EconML models
W = np.delete(X,x_indx, axis=1)

Computing treatment effect heterogeneity with respect to: awscmei


### LinearDML

In [29]:
from econml.dml import LinearDML
ldml = LinearDML(discrete_treatment=True)
ldml.fit(y,T,X=Xh,W=W)

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

Objective did not converge. You might want to increase the number of iterations. Duality gap: 918285.8968609571, tolerance: 180345.44128117117
Objective did not converge. You might want to increase the number of iterations. Duality gap: 4391320.365339518, tolerance: 180345.44128117117
Objective did not converge. You might want to increase the number of iterations. Duality gap: 6364184.545832515, tolerance: 180345.44128117117
Objective did not converge. You might want to increase the number of iterations. Duality gap: 5893209.61648488, tolerance: 180345.44128117117
Objective did not converge. You might want to increase the number of iterations. Duality gap: 5220216.220025301, tolerance: 180345.44128117117
Objective did not converge. You might want to increase the number of iterations. Duality gap: 5214798.51034987, tolerance: 180345.44128117117
Objective did not converge. You might want to increase the number of iterations. Duality gap: 6582379.735553324, tolerance: 180345.44128117117
O

Objective did not converge. You might want to increase the number of iterations. Duality gap: 1829055.8724102378, tolerance: 210248.31160771937
Objective did not converge. You might want to increase the number of iterations. Duality gap: 1728578.1009181738, tolerance: 210248.31160771937
Objective did not converge. You might want to increase the number of iterations. Duality gap: 1928001.5088040233, tolerance: 210248.31160771937
Objective did not converge. You might want to increase the number of iterations. Duality gap: 2518587.2368767858, tolerance: 210248.31160771937
Objective did not converge. You might want to increase the number of iterations. Duality gap: 3870665.406595707, tolerance: 210248.31160771937
Objective did not converge. You might want to increase the number of iterations. Duality gap: 4654514.850140452, tolerance: 210248.31160771937
'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence

Objective did not converge. You might want to increase the number of iterations. Duality gap: 607658.6083658934, tolerance: 174963.6226200908
Objective did not converge. You might want to increase the number of iterations. Duality gap: 702476.1212161183, tolerance: 174963.6226200908
Objective did not converge. You might want to increase the number of iterations. Duality gap: 847160.7584556341, tolerance: 174963.6226200908
Objective did not converge. You might want to increase the number of iterations. Duality gap: 176422.63526898623, tolerance: 171916.08191499257
Objective did not converge. You might want to increase the number of iterations. Duality gap: 186488.8738449812, tolerance: 171916.08191499257
Objective did not converge. You might want to increase the number of iterations. Duality gap: 262353.62865817547, tolerance: 171916.08191499257
Objective did not converge. You might want to increase the number of iterations. Duality gap: 193537.99712324142, tolerance: 171916.08191499257

In [32]:
ldml.ate(X=Xh), ldml.ate_interval(X=Xh)

In [ ]:
from econml.dml import SparseLinearDML
sldml = SparseLinearDML(discrete_treatment=True)
sldml.fit(y,T,X=Xh,W=W)

In [ ]:
sldml.ate(X=Xh), ldml.ate_interval(X=Xh)

### LinearDR Learner

In [ ]:
from econml.dr import LinearDRLearner
ldr = LinearDRLearner()
ldr.fit(y,T,X=Xh,W=W)

In [ ]:
ldr.ate(X=Xh), ldr.ate_interval(X=Xh)

### CausalForestDML

In [ ]:
from econml.dml import CausalForestDML
cf = CausalForestDML(model_y=Lasso(),
                       model_t=LogisticRegression(),
                       discrete_treatment=True,
                       random_state=123)
cf.fit(y, T, X=Xh, W=W)

In [ ]:
cf.ate(X=Xh), cf.ate_interval(X=Xh)

### Meta Learners

In [33]:
from econml.metalearners import TLearner, SLearner, XLearner, DomainAdaptationLearner
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

#### Lasso

In [34]:
models = LassoCV(alphas=np.logspace(-2,4,30),max_iter=5000)
est_t = TLearner(models=models)
est_t.fit(y, T, X=X, inference='bootstrap')

Objective did not converge. You might want to increase the number of iterations. Duality gap: 497055.3242598772, tolerance: 114245.4763408301
Objective did not converge. You might want to increase the number of iterations. Duality gap: 156213.37949693203, tolerance: 114245.4763408301
Objective did not converge. You might want to increase the number of iterations. Duality gap: 159778.52901130915, tolerance: 114245.4763408301
Objective did not converge. You might want to increase the number of iterations. Duality gap: 174565.7763695717, tolerance: 114245.4763408301
Objective did not converge. You might want to increase the number of iterations. Duality gap: 240431.79050028324, tolerance: 114245.4763408301
Objective did not converge. You might want to increase the number of iterations. Duality gap: 682902.4319127798, tolerance: 114245.4763408301
Objective did not converge. You might want to increase the number of iterations. Duality gap: 310851.04122006893, tolerance: 114245.4763408301
Ob

Objective did not converge. You might want to increase the number of iterations. Duality gap: 2178797.707190275, tolerance: 117794.30310608109
Objective did not converge. You might want to increase the number of iterations. Duality gap: 144573.5310086012, tolerance: 126215.42409575291
Objective did not converge. You might want to increase the number of iterations. Duality gap: 152217.82304686308, tolerance: 126215.42409575291
Objective did not converge. You might want to increase the number of iterations. Duality gap: 150769.79483819008, tolerance: 126215.42409575291
Objective did not converge. You might want to increase the number of iterations. Duality gap: 173577.6185696125, tolerance: 126215.42409575291
Objective did not converge. You might want to increase the number of iterations. Duality gap: 148430.02874743938, tolerance: 126215.42409575291
Objective did not converge. You might want to increase the number of iterations. Duality gap: 589051.3300294876, tolerance: 126215.42409575

Objective did not converge. You might want to increase the number of iterations. Duality gap: 14950556.041136503, tolerance: 256784.80832320428
Objective did not converge. You might want to increase the number of iterations. Duality gap: 11997094.25006771, tolerance: 263382.857647637
Objective did not converge. You might want to increase the number of iterations. Duality gap: 437467.47193694115, tolerance: 277598.1807345959
Objective did not converge. You might want to increase the number of iterations. Duality gap: 526936.4038748741, tolerance: 233929.59911444294
Objective did not converge. You might want to increase the number of iterations. Duality gap: 1854874.364278078, tolerance: 248551.21101516753
Objective did not converge. You might want to increase the number of iterations. Duality gap: 382475.0366923809, tolerance: 285795.7877016659
Objective did not converge. You might want to increase the number of iterations. Duality gap: 605369.4963681698, tolerance: 233929.59911444294
O

Objective did not converge. You might want to increase the number of iterations. Duality gap: 3639082.179010868, tolerance: 274912.89036654023
Objective did not converge. You might want to increase the number of iterations. Duality gap: 306780.81842660904, tolerance: 274912.89036654023
Objective did not converge. You might want to increase the number of iterations. Duality gap: 298707.60155916214, tolerance: 291322.4877247124
Objective did not converge. You might want to increase the number of iterations. Duality gap: 1217853.5746426582, tolerance: 246547.5995311163
Objective did not converge. You might want to increase the number of iterations. Duality gap: 263325.17653274536, tolerance: 223965.2037872779
Objective did not converge. You might want to increase the number of iterations. Duality gap: 602891.0280067921, tolerance: 291322.4877247124
Objective did not converge. You might want to increase the number of iterations. Duality gap: 383230.1066119671, tolerance: 274912.89036654023

Objective did not converge. You might want to increase the number of iterations. Duality gap: 186591.82166011631, tolerance: 102211.67978334987
Objective did not converge. You might want to increase the number of iterations. Duality gap: 3091729.665242672, tolerance: 240140.9475524808
Objective did not converge. You might want to increase the number of iterations. Duality gap: 214304.81321166456, tolerance: 102211.67978334987
Objective did not converge. You might want to increase the number of iterations. Duality gap: 277946.60923719406, tolerance: 228854.6148614563
Objective did not converge. You might want to increase the number of iterations. Duality gap: 381366.5790026188, tolerance: 245871.6120551106
Objective did not converge. You might want to increase the number of iterations. Duality gap: 4540465.97652936, tolerance: 240140.9475524808
Objective did not converge. You might want to increase the number of iterations. Duality gap: 229486.63224568963, tolerance: 102211.67978334987


Objective did not converge. You might want to increase the number of iterations. Duality gap: 977065.1758343577, tolerance: 102211.67978334987
Objective did not converge. You might want to increase the number of iterations. Duality gap: 1006630.6184867024, tolerance: 102837.14060188501
Objective did not converge. You might want to increase the number of iterations. Duality gap: 2008302.5859365463, tolerance: 308198.0482736984
Objective did not converge. You might want to increase the number of iterations. Duality gap: 1178689.6680395603, tolerance: 262458.9204991454
Objective did not converge. You might want to increase the number of iterations. Duality gap: 1212270.5134899616, tolerance: 217717.66063110516
Objective did not converge. You might want to increase the number of iterations. Duality gap: 827773.083488971, tolerance: 102837.14060188501
Objective did not converge. You might want to increase the number of iterations. Duality gap: 743629.4363788217, tolerance: 102211.6797833498

Objective did not converge. You might want to increase the number of iterations. Duality gap: 1004991.0898203254, tolerance: 102837.14060188501
Objective did not converge. You might want to increase the number of iterations. Duality gap: 179214.5613437295, tolerance: 116639.13748671193
Objective did not converge. You might want to increase the number of iterations. Duality gap: 5172199.334101945, tolerance: 110776.28535533791
Objective did not converge. You might want to increase the number of iterations. Duality gap: 1166685.1580331475, tolerance: 102837.14060188501
Objective did not converge. You might want to increase the number of iterations. Duality gap: 227367.4706288576, tolerance: 109878.30004976525
Objective did not converge. You might want to increase the number of iterations. Duality gap: 5224031.777174294, tolerance: 110776.28535533791
Objective did not converge. You might want to increase the number of iterations. Duality gap: 1293664.7997797728, tolerance: 102837.14060188

Objective did not converge. You might want to increase the number of iterations. Duality gap: 1044880.9497554302, tolerance: 296473.523329581
Objective did not converge. You might want to increase the number of iterations. Duality gap: 191523.8540199399, tolerance: 114871.78675287464
Objective did not converge. You might want to increase the number of iterations. Duality gap: 185983.8714748025, tolerance: 103951.41995249521
Objective did not converge. You might want to increase the number of iterations. Duality gap: 4186246.1456229687, tolerance: 109878.30004976525
Objective did not converge. You might want to increase the number of iterations. Duality gap: 11779434.067767113, tolerance: 104768.12564869566
Objective did not converge. You might want to increase the number of iterations. Duality gap: 523811.29098027945, tolerance: 122308.64802221154
Objective did not converge. You might want to increase the number of iterations. Duality gap: 314573.6425292492, tolerance: 109729.983464023

KeyboardInterrupt: 

In [35]:
LassoCV?

In [ ]:
est_t.ate(X=X), est_t.ate_interval(X=X)

#### GradientBoosting

In [ ]:
models = GradientBoostingRegressor()
est_t2 = TLearner(models=models)
est_t2.fit(y, T, X=X, inference='bootstrap')

In [ ]:
est_t2.ate(X=X), est_t2.ate_interval(X=X)

In [ ]:
# TODO - figure out how to set the number of bootstrap samples
# TODO - implement manual bootstrapping
# TODO - plot CATE
# TODO - figure out why we have to set X (look at the implementation in EconML)
# TODO - test out some other implementations of DoubleML/CausalForest (maybe in R)
# TODO - look at cricism methods on do-why and test them out (see if we can get the same results through the do-why interface)